In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

In [2]:
train1=pd.read_csv('lightGBM_out_of_fold_train_predictions.csv')

In [3]:
almost_zero = 1e-10
almost_one = 1 - almost_zero

In [4]:
train1['log_lightgbm']=train1['lightGBM_prediction'].clip(almost_zero,almost_one).apply(logit)

In [5]:
train2=pd.read_csv('out_folder_train_prediction_lgb.csv')

In [6]:
train2.head()

,train_pred,SK_ID_CURR,TARGET,folder
0,0.035571,100004,0.0,1
1,0.043243,100006,0.0,1
2,0.051921,100008,0.0,1
3,0.048329,100011,0.0,1
4,0.008429,100018,0.0,1


In [7]:
train2['log_lightgbm']=train2['train_pred'].clip(almost_zero,almost_one).apply(logit)

In [8]:
train3=pd.read_csv('out_folder_train_prediction_xgb.csv')

In [9]:
train3.head()

,train_pred,SK_ID_CURR,TARGET,folder
0,0.035678,100004,0.0,1
1,0.042576,100006,0.0,1
2,0.057968,100008,0.0,1
3,0.055486,100011,0.0,1
4,0.006375,100018,0.0,1


In [10]:
train3['log_lightgbm']=train3['train_pred'].clip(almost_zero,almost_one).apply(logit)

In [11]:
dfs = [train1,train2,train3]

In [12]:
train_final = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)

In [13]:
train_final.head()

,SK_ID_CURR,fold_id,lightGBM_prediction,log_lightgbm_x,train_pred_x,TARGET_x,folder_x,log_lightgbm_y,train_pred_y,TARGET_y,folder_y,log_lightgbm
0,100002,0,0.304156,-0.827585,0.286197,1.0,2,-0.913927,0.283627,1.0,2,-0.926543
1,100009,0,0.005594,-5.180468,0.004515,0.0,3,-5.395821,0.005745,0.0,3,-5.153730
2,100011,0,0.049608,-2.952730,0.048329,0.0,1,-2.980189,0.055486,0.0,1,-2.834533
3,100014,0,0.045775,-3.037156,0.042180,0.0,2,-3.122711,0.042405,0.0,2,-3.117166
4,100017,0,0.018538,-3.969205,0.024226,0.0,3,-3.695810,0.021061,0.0,3,-3.839025


In [14]:
X_train=train_final[['log_lightgbm_x','log_lightgbm_y','log_lightgbm' ]]
y_train=train_final['TARGET_x']

In [15]:
stack_model = LogisticRegression()
stack_model.fit(X_train, y_train)
stack_model.coef_

array([[0.59430332, 0.26110589, 0.16914241]])

In [16]:
stack_model.score(X_train, y_train)

0.9202597630653863

In [17]:
test1=pd.read_csv('lightGBM_out_of_fold_test_predictions.csv')

In [18]:
test1.head()

,SK_ID_CURR,fold_id,lightGBM_prediction
0,100001,0,0.046421
1,100005,0,0.113959
2,100013,0,0.036334
3,100028,0,0.046933
4,100038,0,0.159083


In [19]:
test1['log_lightgbm']=test1['lightGBM_prediction'].clip(almost_zero,almost_one).apply(logit)

In [20]:
test2=pd.read_csv('out_folder_test_prediction_lgb.csv')

In [21]:
test2.head()

,SK_ID_CURR,test_pred,folder
0,100001,0.028859,1
1,100005,0.171567,1
2,100013,0.035817,1
3,100028,0.041928,1
4,100038,0.151693,1


In [22]:
test2['log_lightgbm']=test2['test_pred'].clip(almost_zero,almost_one).apply(logit)

In [23]:
test3=pd.read_csv('out_folder_test_prediction_xgb.csv')

In [24]:
test3['log_lightgbm']=test3['test_pred'].clip(almost_zero,almost_one).apply(logit)

In [25]:
dfs = [test1[test1['fold_id']==0],test2[test2['folder']==1],test3[test3['folder']==1]]

In [26]:
test_final1 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)

In [27]:
test_final1.head()

,SK_ID_CURR,fold_id,lightGBM_prediction,log_lightgbm_x,test_pred_x,folder_x,log_lightgbm_y,test_pred_y,folder_y,log_lightgbm
0,100001,0,0.046421,-3.022473,0.028859,1,-3.516059,0.029790,1,-3.483333
1,100005,0,0.113959,-2.050926,0.171567,1,-1.574564,0.237822,1,-1.164660
2,100013,0,0.036334,-3.277997,0.035817,1,-3.292869,0.027966,1,-3.548411
3,100028,0,0.046933,-3.010966,0.041928,1,-3.128976,0.035709,1,-3.295997
4,100038,0,0.159083,-1.665063,0.151693,1,-1.721386,0.181272,1,-1.507754


In [28]:
pred0=stack_model.predict_proba(test_final1[['log_lightgbm_x','log_lightgbm_y','log_lightgbm' ]])[:,1]

In [29]:
dfs = [test1[test1['fold_id']==1],test2[test2['folder']==2],test3[test3['folder']==2]]
test_final2 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred1=stack_model.predict_proba(test_final2[['log_lightgbm_x','log_lightgbm_y','log_lightgbm']])[:,1]

In [30]:
dfs = [test1[test1['fold_id']==2],test2[test2['folder']==3],test3[test3['folder']==3]]
test_final3 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred2=stack_model.predict_proba(test_final3[['log_lightgbm_x','log_lightgbm_y','log_lightgbm' ]])[:,1]

In [31]:
dfs = [test1[test1['fold_id']==3],test2[test2['folder']==4],test3[test3['folder']==4]]
test_final4 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred3=stack_model.predict_proba(test_final4[['log_lightgbm_x','log_lightgbm_y','log_lightgbm']])[:,1]

In [32]:
dfs = [test1[test1['fold_id']==4],test2[test2['folder']==5],test3[test3['folder']==5]]
test_final5 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred4=stack_model.predict_proba(test_final5[['log_lightgbm_x','log_lightgbm_y','log_lightgbm' ]])[:,1]

In [33]:
sub = pd.DataFrame()
sub['SK_ID_CURR'] = test1[test1['fold_id']==0]['SK_ID_CURR']

In [34]:
sub['pred0']=pred0
sub['pred1']=pred1
sub['pred2']=pred2
sub['pred3']=pred3
sub['pred4']=pred4

In [35]:
sub['TARGET']=(sub['pred0']+sub['pred1']+sub['pred2']+sub['pred3']+sub['pred4'])/5

In [36]:
np.min(sub['TARGET'])

0.0017871313891496155

In [37]:
np.max(sub['TARGET'])

0.7716428200235529

In [38]:
sub[['SK_ID_CURR', 'TARGET']].to_csv('logit_stacking_model.csv', index= False)